<a href="https://colab.research.google.com/github/OsvaldoFloresC/OsvaldoFloresC/blob/main/Pivoteo_Parcial_Escalado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Eliminación gaussiana con pivoteo parcial escalado.**
El pivoteo parcial escalado es una mejora adicional del pivoteo parcial. Antes de seleccionar el pivote, cada fila de la matriz se escala por el valor absoluto más grande en esa fila. Esto asegura que la selección del pivote sea menos sensible a las variaciones en la magnitud de los coeficientes, proporcionando una mayor estabilidad numérica.

Ventajas del Pivoteo Parcial Escalado

Mayor Estabilidad Numérica: El escalamiento ayuda a evitar divisiones por números muy pequeños, lo que reduce los errores de redondeo y mejora la precisión de la solución.

Selección de Pivotes Más Eficiente: Al escalar las filas antes de seleccionar el pivote, se asegura que se elija el mejor pivote posible en términos relativos.

Aplicable a Sistemas Mal Condicionados: Es especialmente útil para resolver sistemas de ecuaciones que son mal condicionados o tienen coeficientes con magnitudes muy diferentes.

In [ ]:
import numpy as np

def eliminacion_gaussiana_pivoteo_escalado(A, b, cifras_significativas=3):

    def redondear_a_cifras_significativas(x, cifras_sig):
        """Redondeo a cifras significativas."""
        return np.round(x, cifras_sig - int(np.floor(np.log10(abs(x)))) - 1) if x != 0 else 0

    n = len(b)
    A = A.copy()
    b = b.copy()

    # Redondear la matriz inicial y el vector independiente a cifras significativas
    A = np.vectorize(lambda x: redondear_a_cifras_significativas(x, cifras_significativas))(A)
    b = np.vectorize(lambda x: redondear_a_cifras_significativas(x, cifras_significativas))(b)

    # Extender la matriz A con b para realizar las operaciones simultáneamente
    Ab = np.hstack((A, b.reshape(-1, 1)))

    # Imprimir la matriz inicial
    print("Matriz Aumentada Inicial:")
    print(Ab)

    # Paso 1: Escalamiento y determinación inicial
    s = np.max(np.abs(A), axis=1)  # Vector de escalas
    if np.any(s == 0):
        raise ValueError("No existe solución única: escala cero detectada.")

    # Vector de índices que representa el orden de las filas
    NROW = np.arange(n)

    # Proceso de eliminación
    for i in range(n - 1):
        # Paso 2: Determinación del pivote escalado
        ratios = np.abs(Ab[NROW[i:], i]) / s[NROW[i:]]
        p = i + np.argmax(ratios)  # Índice del máximo elemento escalado

        # Intercambio de filas
        if p != i:
            NROW[i], NROW[p] = NROW[p], NROW[i]

        # Paso 3: Eliminación hacia adelante
        for j in range(i + 1, n):
            factor = redondear_a_cifras_significativas(Ab[NROW[j], i] / Ab[NROW[i], i], cifras_significativas)
            Ab[NROW[j], i:] = np.vectorize(lambda x: redondear_a_cifras_significativas(x, cifras_significativas))(
                Ab[NROW[j], i:] - factor * Ab[NROW[i], i:]
            )

    # Sustitución hacia atrás
    x = np.zeros(n)
    for i in range(n - 1, -1, -1):
        x[i] = redondear_a_cifras_significativas(
            (Ab[NROW[i], -1] - np.dot(Ab[NROW[i], i + 1:n], x[i + 1:n])) / Ab[NROW[i], i],
            cifras_significativas
        )

    return x


# Datos del sistema (matriz de coeficientes y vector independiente)
A = np.array([
    [2.11, -4.21, 0.921],
    [4.01, 10.2, -1.12],
    [1.09, 0.987, 0.832]
])

b = np.array([2.01, -3.09, 4.21])

# Resolviendo el sistema ajustando a 3 cifras significativas
solucion_redondeada = eliminacion_gaussiana_pivoteo_escalado(A, b, cifras_significativas=3)

# Mostrando la solución
print("\nSolución del sistema:")
for i, val in enumerate(solucion_redondeada, start=1):
    print(f"x{i} = {val:.3f}")


Matriz Aumentada Inicial:
[[ 2.11  -4.21   0.921  2.01 ]
 [ 4.01  10.2   -1.12  -3.09 ]
 [ 1.09   0.987  0.832  4.21 ]]

Solución del sistema:
x1 = -0.432
x2 = 0.427
x3 = 5.120


La eliminación gaussiana con pivoteo parcial escalado es una técnica mejorada para resolver sistemas de ecuaciones lineales que combina dos enfoques para mejorar la estabilidad numérica y la precisión de las soluciones